### Chat Básico

**Objetivo:** Receber uma mensagem do usuário e gerar uma resposta com LLM.

Esse é o caso mais básico de uso de LLM em LangGraph.

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph.message import add_messages

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from typing import Annotated, TypedDict

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
def chatbot(state: State):
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

In [ ]:
builder = StateGraph(State)

builder.add_node("chatbot", chatbot)

builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

graph = builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
response = graph.invoke({
    "messages": [
        HumanMessage(content="Qual é a capital de Pernambuco?"),
    ]
})

In [ ]:
for msg in response["messages"]:
    print(f"{msg.type}: {msg.content}")

### Ferramentas

In [ ]:
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    """Retorna a previsão do tempo para uma localização específica."""
    return f"A previsão do tempo em {location} é ensolarada com uma temperatura de 25°C."

tools = [get_weather]

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
result = llm_with_tools.invoke("qual é a previsão do tempo em Natal?")
print(result)

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition

def chatbot(state: State):
    messages = state["messages"]
    return {"messages": [llm_with_tools.invoke(messages)]}

tool_node = ToolNode(tools=tools)

In [ ]:
builder = StateGraph(State)

builder.add_node("chatbot", chatbot)
builder.add_node("tools", tool_node)

builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

builder.add_edge("tools", "chatbot")
builder.add_edge(START, "chatbot")

graph = builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
response = graph.invoke({
    "messages": [
        HumanMessage(content="Qual o clima em Recife?"),
    ]
})

for msg in response["messages"]:
    print(f"{msg.type}: {msg.content}")